### Some initial preparations

We install the brainscapes library, and set an EBRAINS Knowledge Graph authentication token the we can request  [here](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize) after creating an HBP account.

In [1]:
!pip install brainscapes
from os import environ
environ['HBP_AUTH_TOKEN'] = "<<<Paste your HBP auth token here>>>"

zsh:1: command not found: pip


### 1. Instantiate human brain atlas with Julich-Brain cytoarchitectonic maps

We instantiate the human atlas, and tell it to prefer thresholded continous maps over labelled regions in the discrete parcellation map. For the Julich-Brain parcellation, this mean that it will use thresholded probability maps, if available.

In [1]:
import brainscapes as bs
bs.logger.setLevel("INFO") # we want to see some messages!

In [2]:
atlas = bs.atlases.MULTILEVEL_HUMAN_ATLAS

# next line is optional - cytoarchitectonic maps are selected by default
atlas.select_parcellation(bs.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_5_)

# as in the original JuGEx, we prefer thresholded probability maps 
# over the labelled region in the maximum probability map
atlas.enable_continuous_map_thresholding(0.2)

[brainscapes:INFO]  Selected parcellation "Julich-Brain Probabilistic Cytoarchitectonic Maps (v2.5)"


### 2. Instantiate a differential gene expression analysis with this atlas

The analysis object will check if the parcellation selected in the atlas is suitable for the analysis. What it does in fact, is to verify that it provides maps in the MNI ICBM 152 space. We add a few candidate genes that we're interested in. They could also be added right away as a list to the constructor in the first line below. Then we select two regions of interest (ROI) by specifying brain area names that the atlas can interpret.

In [3]:
jugex = bs.analysis.DifferentialGeneExpression(atlas)
jugex.add_candidate_gene(bs.features.gene_names.MAOA)
jugex.add_candidate_gene(bs.features.gene_names.TAC1)
jugex.define_roi1("v1 right")
jugex.define_roi2("v2 right")

[brainscapes:INFO]  Selected region Area hOc1 (V1, 17, CalcS) - right hemisphere
[brainscapes:INFO]  Retrieving probe ids for gene MAOA
[brainscapes:INFO]  Using continuous map thresholded by 0.2 for masking region Area hOc1 (V1, 17, CalcS) - right hemisphere.
[brainscapes:INFO]  Retrieving probe ids for gene TAC1
[brainscapes:INFO]  13 samples found for region v1 right.
[brainscapes:INFO]  Selected region Area hOc2 (V2, 18) - right hemisphere
[brainscapes:INFO]  Retrieving probe ids for gene MAOA
[brainscapes:INFO]  Using continuous map thresholded by 0.2 for masking region Area hOc2 (V2, 18) - right hemisphere.
[brainscapes:INFO]  Retrieving probe ids for gene TAC1
[brainscapes:INFO]  13 samples found for region v2 right.


### 3. Look at the samples

In [5]:
jugex.samples1

defaultdict(dict,
            {(11.124800000000008,
              -70.751,
              15.294900000000005): {'race': 'Black or African American', 'gender': 'M', 'age': 24, 'name': 'H0351.2001', 'id': 9861, 'area': 'Area hOc1 (V1, 17, CalcS) - right hemisphere', 'MAOA': -0.08148064516129032, 'TAC1': -0.5691076923076923},
             (7.2579000000000065,
              -98.01410000000001,
              7.96520000000001): {'race': 'Black or African American', 'gender': 'M', 'age': 24, 'name': 'H0351.2001', 'id': 9861, 'area': 'Area hOc1 (V1, 17, CalcS) - right hemisphere', 'MAOA': 0.25943548387096776, 'TAC1': -0.5396},
             (7.040199999999999,
              -65.2882,
              2.959500000000002): {'race': 'Black or African American', 'gender': 'M', 'age': 24, 'name': 'H0351.2001', 'id': 9861, 'area': 'Area hOc1 (V1, 17, CalcS) - right hemisphere', 'MAOA': -0.5113903225806451, 'TAC1': -0.514223076923077},
             (29.423100000000005,
              -56.66350000000003,
   

### 4. Run the analysis

We are now setup to run the analysis.

In [4]:
jugex.run(permutations=1000,random_seed=42)
print(jugex.result)

[brainscapes:INFO]  Using random seed 42.
[brainscapes:INFO]  Running 1000 random permutations. This may take a while...


{'MAOA': 0.572, 'TAC1': 0.076}
